# Setting the reference point

In [2]:
import cv2
import pyrealsense2

from realsense_depth import *  #note: I have changed the frame size to 848X480 from 640X480



def empty(a):
    pass

#for displaying trackbar to adjust values of kernal,threshold and min area of contour
cv2.namedWindow("Parameter")
cv2.resizeWindow("Parameter",640,240)
cv2.createTrackbar("kernel","Parameter",7,50,empty)
cv2.createTrackbar("Threshold","Parameter",75,254,empty)
cv2.createTrackbar("AREA","Parameter",250,9000,empty)

# Initialize Camera Intel Realsense
dc = DepthCamera()

#capturing backgroung image
first_frame= True

while first_frame is True:
    ret, depth_bg_frame, color_bg_frame = dc.get_frame()
    bg_bilateral = cv2.bilateralFilter(color_bg_frame, 15, 75, 75)
    bg_gray = cv2.cvtColor(bg_bilateral, cv2.COLOR_BGR2GRAY)
    cv2.imshow("showing Video for bg",bg_gray)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        ff=bg_gray
        #cv2.imwrite("c1.png",bg_gray)
        first_frame= False





# Create mouse event
#cv2.namedWindow("Color frame")
#cv2.setMouseCallback("Color frame", show_distance)

while True:
    ret, depth_frame, color_frame = dc.get_frame()
    if ret is True:
        bilateral = cv2.bilateralFilter(color_frame, 15, 60, 60)
        gray = cv2.cvtColor(bilateral, cv2.COLOR_BGR2GRAY)
        image_sub = cv2.absdiff(ff, gray)
        #kernel=cv2.getTrackbarPos("kernel", "Parameter")#for trackbar
        #if (kernel%2) ==0:
        #    kernel=kernel+1
        #else:
        #    pass
        #blur=cv2.medianBlur(gray,kernel)
        thres=cv2.getTrackbarPos("Threshold","Parameter")#for trackbar
        retu, th1 = cv2.threshold(image_sub,thres,255,cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(color_frame, contours, -1, (0,255,0), 1)
        for i in range (len(contours)):
            cnt2 = contours[i]
            area=cv2.contourArea(cnt2)
            AREA = cv2.getTrackbarPos("AREA", "Parameter")#for trackbar
            if area>=AREA:
                #epsilon = 0.1*cv2.arcLength(cnt2,True)
                #approx = cv2.approxPolyDP(cnt2,epsilon,True)
                rect = cv2.minAreaRect(cnt2) #returns [rectangle center(x, y), (width, height), rotation angle)].
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                color_frame = cv2.drawContours(color_frame,[box],0,(0,255,255),2)
                #M = cv2.moments(cnt2)
                #cx2 = int(M['m10']/M['m00'])
                #cy2 = int(M['m01']/M['m00'])
                cx2=int(rect[0][0])
                cy2=int(rect[0][1])
                point = (cx2 , cy2)
                
                width= int(rect[1][0])
                height= int(rect[1][1])
                
                 # Show distance for a specific point
                cv2.circle(color_frame, point, 4, (0, 0, 255))
                distance =depth_frame[int(point[1]),int(point[0])] #depth_frame[240,320] #
                
                dist=(distance)/10          #input in cm
                cv2.putText(color_frame, "{}cm".format(dist), (point[0], point[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        
                #cvtfac=0.00304535*distance**(0.921814) -0.17026#1#0.00167228*distance -0.00381802
                cvtfac =1#0.00325729*(distance)**(0.914301)-0.144273
                #Rx=(round(width*cvtfac,2))
                #H8
                #Ry=(round(height*cvtfac,2))
                #print(width,height)
                
                #Pixel_to_CM
                
                Rx=(round(cx2*cvtfac,2))
                Ry=(round(cy2*cvtfac,2))
                
                
                #color_frame=cv2.circle(color_frame, (cx2, cy2), 7, (0, 100, 200), -1)
                angle=rect[-1]
                #print(angle)
                if angle<-45:
                    angle=-(90+angle)
                else:
                    angle=-angle
                #print(angle,"deg")
                x_dis=abs(cx2-box[0,0]) #dist btw x value of center of contour and x-value of first vertex of contour
                y_dis=abs(cy2-box[0,1])#dist btw y value of center of contour and y-value of first vertex of contour
                
                ###########################################
                #below values are not used because we change the way to detect orientation
                #a=np.array((box[0,0],box[0,1]))
                #b=np.array((box[1,0],box[1,1]))
                #c=np.array((box[2,0],box[2,1]))
                #ab=np.linalg.norm(a-b)
                #bc=np.linalg.norm(b-c)
                #print(ab,bc)
                
                #if (ab<bc and (angle<=0 and angle>-10)) or (ab>bc and (angle>0 and angle<10)):
                 #   frame=cv2.putText(frame,"Vertical" , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    #print("its verticle")
                #elif (ab>bc and (angle<=0 and angle>-10)) or (ab<bc and (angle>0 and angle<10)):
                    #print("its Horizontal")
                 #   frame=cv2.putText(frame,"Horizontal" , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                #############################################
                
               
                if (x_dis<y_dis and (angle<10 and angle>-10)):
                    color_frame=cv2.putText(color_frame,"v"+str( Rx)+","+str( Ry) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    #print("its verticle")
                elif (x_dis>y_dis and (angle<10 and angle>-10)):
                    #print("its Horizontal")
                    color_frame=cv2.putText(color_frame,"H"+str(Rx)+","+str(Ry), (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                else:
                    color_frame=cv2.putText(color_frame,"I"+str(Rx)+","+str(Ry) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (155, 0, 0), 2)
                    #print("Its inclined")

        
    else:
        break

    #cv2.imshow("depth frame", depth_frame)
    cv2.imshow("Color frame", color_frame)
    #cv2.imshow("gray frame", gray)
    #cv2.imshow("blur frame", blur)
    #cv2.imshow("th1 frame", th1)
    key = cv2.waitKey(1)
    if key == 27:
        dc.release()
        break
cv2.destroyAllWindows()

# getting the actual location of the box

In [ ]:
import cv2
import pyrealsense2
from pylogix import PLC
import time
from realsense_depth import *



def empty(a):
    pass

def Sort(s):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    s.sort(key = lambda x: x[0])
    return s


#for displaying trackbar to adjust values of kernal,threshold and min area of contour
cv2.namedWindow("Parameter")
cv2.resizeWindow("Parameter",640,240)
cv2.createTrackbar("kernel","Parameter",7,50,empty)
cv2.createTrackbar("Threshold","Parameter",50,254,empty)
cv2.createTrackbar("AREA","Parameter",4900,9000,empty)

# Initialize Camera Intel Realsense
dc = DepthCamera()

#capturing backgroung image
first_frame= True

while first_frame is True:
    ret, depth_bg_frame, color_bg_frame = dc.get_frame()
    bg_bilateral = cv2.bilateralFilter(color_bg_frame, 15, 75, 75)
    bg_gray = cv2.cvtColor(bg_bilateral, cv2.COLOR_BGR2GRAY)
    cv2.imshow("showing Video for bg",bg_gray)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        ff=bg_gray
        #cv2.imwrite("c1.png",bg_gray)
        first_frame= False

nf_color=False
nf_depth=False
arr=np.array([])
while True:
    ret, depth_frame, color_frame = dc.get_frame()
    if cv2.waitKey(1) & 0xFF == ord('c'):
        arr=[]
        nf_color=color_frame
        nf_depth=depth_frame
        bilateral = cv2.bilateralFilter(nf_color, 15, 60, 60)
        gray = cv2.cvtColor(bilateral, cv2.COLOR_BGR2GRAY)
        image_sub = cv2.absdiff(ff, gray)
        #kernel=cv2.getTrackbarPos("kernel", "Parameter")#for trackbar
        #if (kernel%2) ==0:
        #    kernel=kernel+1
        #else:
        #    pass
        #blur=cv2.medianBlur(gray,kernel)
        thres=cv2.getTrackbarPos("Threshold","Parameter")#for trackbar
        retu, th1 = cv2.threshold(image_sub,thres,255,cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(nf_color, contours, -1, (0,255,0), 1)
        for i in range (len(contours)):
            cnt2 = contours[i]
            area=cv2.contourArea(cnt2)
            AREA = cv2.getTrackbarPos("AREA", "Parameter")#for trackbar
            if area>=AREA:
                #epsilon = 0.1*cv2.arcLength(cnt2,True)
                #approx = cv2.approxPolyDP(cnt2,epsilon,True)
                rect = cv2.minAreaRect(cnt2) #returns [rectangle center(x, y), (width, height), rotation angle)].
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                color_frame = cv2.drawContours(nf_color,[box],0,(0,255,255),2)
                #M = cv2.moments(cnt2)
                #cx2 = int(M['m10']/M['m00'])
                #cy2 = int(M['m01']/M['m00'])
                cx2=int(rect[0][0])
                cy2=int(rect[0][1])
                point = (cx2 , cy2)

                width= int(rect[1][0])
                height= int(rect[1][1])

                 # Show distance for a specific point
                cv2.circle(nf_color, point, 4, (0, 0, 255))
                distance = nf_depth[240, 320] # [point[1],point[0]]

                dist=distance/10          #input in cm
                cv2.putText(nf_color, "{}cm".format(dist), (point[0], point[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

                cvtfac=0.00167228*distance -0.00381802
                WT=(round(width*cvtfac,2))
                H8=(round(height*cvtfac,2))
                #print(width,height)

                #Pixel_to_MM

                Rx=(round(cx2*cvtfac,2))
                Ry=(round(cy2*cvtfac,2))


                #color_frame=cv2.circle(color_frame, (cx2, cy2), 7, (0, 100, 200), -1)
                angle=rect[-1]
                #print(angle)
                if angle<-45:
                    angle=-(90+angle)
                else:
                    angle=-angle
                #print(angle,"deg")
                x_dis=abs(cx2-box[0,0]) #dist btw x value of center of contour and x-value of first vertex of contour
                y_dis=abs(cy2-box[0,1])#dist btw y value of center of contour and y-value of first vertex of contour

                ###########################################
                #below values are not used because we change the way to detect orientation
                #a=np.array((box[0,0],box[0,1]))
                #b=np.array((box[1,0],box[1,1]))
                #c=np.array((box[2,0],box[2,1]))
                #ab=np.linalg.norm(a-b)
                #bc=np.linalg.norm(b-c)
                #print(ab,bc)

                #if (ab<bc and (angle<=0 and angle>-10)) or (ab>bc and (angle>0 and angle<10)):
                 #   frame=cv2.putText(frame,"Vertical" , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    #print("its verticle")
                #elif (ab>bc and (angle<=0 and angle>-10)) or (ab<bc and (angle>0 and angle<10)):
                    #print("its Horizontal")
                 #   frame=cv2.putText(frame,"Horizontal" , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                #############################################


                if (x_dis<y_dis and (angle<10 and angle>-10)):
                    nf_color=cv2.putText(nf_color,"v"+str( WT)+","+str( H8) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    orientation=900
                    #print("its verticle")
                elif (x_dis>y_dis and (angle<10 and angle>-10)):
                    orientation=0
                    #print("its Horizontal")
                    nf_color=cv2.putText(nf_color,"H"+str(WT)+","+str(H8), (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                else:
                    nf_color=cv2.putText(nf_color,"I"+str( WT)+","+str(H8) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (155, 0, 0), 2)
                    orientation=450
                    #print("Its inclined")
                #To ROBOT
                robx=(round((cx2-320)*cvtfac,0))
                roby=(round((240-cy2)*cvtfac,0))
                 
                
                arr.append([robx,roby, -1710+distance,0,0 ,orientation])
        cv2.imshow("new frame Color", nf_color)
        key = cv2.waitKey(0)
        nf_color=False
        nf_depth=False
        print(arr)
        arr=Sort(arr)
        print(arr)
        for i in range(len(arr)):
            comm =PLC()
            comm.IPAddress = '192.168.255.2'
            n=comm.Read('pulse_to_pc')
            print(n.Value)
            real_x=arr[i][0]
            real_y=arr[i][1]
            real_z=arr[i][2]
            real_rx=arr[i][3]
            real_ry=arr[i][4]
            real_rz=arr[i][5]
            print("Grab item from x:" +str(real_y)+", y:"+str(real_x)+", z:"+str(real_z)+", Rz:"+str(real_rz))
            comm.IPAddress = '192.168.255.2'
            comm.Write('x_axis_frm_pc', real_y)
            comm.Write('y_axis_frm_pc', real_x)
            comm.Write('z_axis_frm_pc', real_z)
            comm.Write('r_x_frm_pc', real_rx)
            comm.Write('r_y_frm_pc', real_ry)
            comm.Write('r_z_frm_pc', real_rz)
            comm.Close()
            #time.sleep(20)
            while n.Value is False:
                print(n.Value)
                n=comm.Read('pulse_to_pc')# should get pulse of '1'
                print(n.Value)
            
    else:
        pass
    cv2.imshow("Color frame", color_frame)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('k'):
        break
cv2.destroyAllWindows()







[]
[]
[]
[]
[[243.0, 2.0, -296, 0, 0, 0]]
[[243.0, 2.0, -296, 0, 0, 0]]
False
Grab item from x:2.0, y:243.0, z:-296, Rz:0
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [1]:
import cv2
arr=[1,3,4,5]
i=0
for i in range(len(arr)):
    a=1
    s=2
    print(i)
    while s==2:
        s=1
        pass

0
1
2
3


In [19]:
def Sort(s):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    s.sort(key = lambda x: x[1])
    print(s)
    s.sort(key = lambda x: x[0])
    return s
  
# Driver Code
arr=[[33,2,33],[55,3,76],[32,76,73],[95,1,22],[11,2,11]]
print(Sort(arr))

[[95, 1, 22], [33, 2, 33], [11, 2, 11], [55, 3, 76], [32, 76, 73]]
[[11, 2, 11], [32, 76, 73], [33, 2, 33], [55, 3, 76], [95, 1, 22]]


In [9]:
import numpy as np
a=[2,4,5,6]
s=a*[[2],
     [4],
     [5],
     [6]]

TypeError: can't multiply sequence by non-int of type 'list'

In [2]:
print(s)

[ 4 16 25 36]
